In [1]:
#COMMIT DA AULA 3

import pandas as pd 
from modules.operations import Operations as op
from modules.specific_operations import SpecificOperations as sop
from modules.config import Config as config
from modules.bibtex_reader import BibtexReader 
from modules.folder_reader import FolderReader
from modules.sqlite3_loader import Sqlite3Handler
from modules.ieee_api_handler import IeeeApiHandler
from modules.science_direct_api_handler import ScienceDirectApiHandler

OUTPUT_FOLDER = './output'
CONFIG_FILEPATH = './config.yaml'

JCS_INPUT_FILEPATH = "./data/jcs_2020.csv"
SCIMAGO_INPUT_FILEPATH = "./data/scimagojr 2020.csv"

ACM_INPUT_FOLDER = "./data/acm"
IEEE_INPUT_FOLDER = "./data/ieee"
SCIENCE_INPUT_FOLDER = "./data/science_direct"

DB_FILEPATH = ':memory:'
OUTPUT_DB_FILEPATH = './output/db.sqlite3'

CONFIG = config.get_config(CONFIG_FILEPATH)

# Extração

In [ ]:
DF_ACM = BibtexReader.read_files_to_dataframe(
    FolderReader.get_filepaths_from_folder(ACM_INPUT_FOLDER)
)

DF_IEEE = BibtexReader.read_files_to_dataframe(
    FolderReader.get_filepaths_from_folder(IEEE_INPUT_FOLDER)
)

DF_SD = BibtexReader.read_files_to_dataframe(
    FolderReader.get_filepaths_from_folder(SCIENCE_INPUT_FOLDER)
)

DF_JCS = pd.read_csv(JCS_INPUT_FILEPATH, sep=";")

DF_SCIMAGO = pd.read_csv(SCIMAGO_INPUT_FILEPATH, sep=";", low_memory=False)

# Transformação - Tratamento dos bibtex

In [ ]:
df_acm = (DF_ACM
          .convert_dtypes()
          .pipe(op.map, ["author", "title", "keywords", "abstract", "year", "ENTRYTYPE", "doi", "issn", "isbn", "journal"])
          .pipe(op.create_column, "source", str, "acm")
          .pipe(op.rename_cols, {
                "ENTRYTYPE": "type_publication"
          })
          .pipe(op.apply_to_every_row, function = lambda r: pd.NA if pd.isnull(r.issn) else r.issn.replace("-", ""),
                                         output_col = 'issn'))

df_ieee = (DF_IEEE
          .convert_dtypes()  
          .pipe(op.map, ["author", "title", "keywords", "abstract", "year", "ENTRYTYPE", "doi", "issn", "journal"])
          .pipe(op.create_column, "isbn", str, pd.NA)
          .pipe(op.create_column, "source", str, "ieee")
          .pipe(op.rename_cols, {
                "ENTRYTYPE": "type_publication"
          })
          .pipe(op.apply_to_every_row, function = lambda r: pd.NA if pd.isnull(r.issn) else r.issn.replace("-", ""),
                                       output_col = 'issn'))

df_sd = (DF_SD
          .convert_dtypes()  
          .pipe(op.map, ["author", "title", "keywords", "abstract", "year", "ENTRYTYPE", "doi", "issn", "isbn", "journal"])
          .pipe(op.create_column, "source", str, "science direct")
          .pipe(op.rename_cols, {
                "ENTRYTYPE": "type_publication"})
          .pipe(op.apply_to_every_row, function = lambda r: pd.NA if pd.isnull(r.issn) else r.issn.replace("-", ""),
                                         output_col = 'issn'))

# Transformação - JCS e SCIMAGO

In [ ]:
# União de todos os arquivos bibtex em um só Dataframe
df_bibtex = (pd.concat([df_acm, df_ieee, df_sd])
            .convert_dtypes()
            .drop_duplicates()
            .pipe(op.apply_to_every_row, 
                  function = lambda r: pd.NA if pd.isnull(r.journal) else r.journal.upper().replace(r"([^A-Za-z0-9]+)", "").strip(),
                  output_col = 'upper_journal'))


# União dos arquivos scimago e jcr em um só Dataframe
df_scimago = (
      DF_SCIMAGO
      .convert_dtypes()
      .pipe(op.map, cols =['Issn', 'Title', 'SJR'])
      .pipe(op.rename_cols, cols = {
            'SJR': 'scimago_value',
            'Issn': 'issn',
            'Title': 'title'
      })
      .pipe(op.apply_to_every_row,  
            function = lambda r: r.title.upper().replace(r"([^A-Za-z0-9]+)", "").strip(), 
            output_col = 'upper_title')
            
      .pipe(op.convert_type, col = 'scimago_value', type = str)
      .pipe(op.apply_to_every_row,  
            function = lambda r: r.scimago_value.replace(",", "."),
            output_col = 'scimago_value'))

df_jcs = (
      DF_JCS
      .convert_dtypes()  
      .pipe(op.map, cols = ["Full Journal Title", "Journal Impact Factor"])
      .pipe(op.rename_cols, cols = {
            "Full Journal Title": "title",
            "Journal Impact Factor": "jcs_value"
      })
      .pipe(op.apply_to_every_row,  
            function = lambda r: r.title.upper().replace(r"([^A-Za-z0-9]+)", "").strip(),
            output_col = 'upper_title' ))

df_journal = (
      df_scimago
      .pipe(op.join, df_right = df_jcs, 
                        left_on = ["upper_title"],
                        right_on = ["upper_title"],
                        how = "outer") 
      .pipe(op.map, cols = ['issn', 'upper_title', 'scimago_value', 'jcs_value']) 
      .drop_duplicates()
      .pipe(op.rename_cols, cols = {
            'issn': 'issn_journal'
      })
      .pipe(op.apply_to_every_row,  
            function = lambda r: '-' if pd.isnull(r.issn_journal) else r.issn_journal,
            output_col = 'issn_journal'))

df_regex_groups = df_journal['issn_journal'].str.split(pat=",", n=3, expand=True)
df_journal['issn_1'] = df_regex_groups.loc[:, 0]
df_journal['issn_2'] = df_regex_groups.loc[:, 1]
df_journal['issn_3'] = df_regex_groups.loc[:, 2]

df_journal = (
      df_journal
      .pipe(op.apply_to_every_row,  
            function = lambda r: pd.NA if r.issn_1 == '-' else r.issn_1,
            output_col = 'issn_1')
      .pipe(op.apply_to_every_row,  
            function = lambda r: pd.NA if r.issn_2 == '-' else r.issn_2,
            output_col = 'issn_2')
      .pipe(op.apply_to_every_row,  
            function = lambda r: pd.NA if r.issn_3 == '-' else r.issn_3,
            output_col = 'issn_3')
      .drop_duplicates())

# Joins entre os bibtex e jcr/scimago
CONNECTION = Sqlite3Handler.get_connection(DB_FILEPATH)
Sqlite3Handler.load_dataframe_to_db(df_bibtex, connection = CONNECTION, table_name = 'df_bibtex', if_exists = 'replace')
Sqlite3Handler.load_dataframe_to_db(df_journal, connection = CONNECTION, table_name = 'df_journal', if_exists = 'replace')
Sqlite3Handler.create_index(CONNECTION, "idx_df_bibtex_1", "df_bibtex", ["issn"])
Sqlite3Handler.create_index(CONNECTION, "idx_df_bibtex_2", "df_bibtex", ["upper_journal"])
Sqlite3Handler.create_index(CONNECTION, "idx_df_journal_1", "df_journal", ["issn_1"])
Sqlite3Handler.create_index(CONNECTION, "idx_df_journal_2", "df_journal", ["issn_2"])
Sqlite3Handler.create_index(CONNECTION, "idx_df_journal_3", "df_journal", ["issn_3"])
Sqlite3Handler.create_index(CONNECTION, "idx_df_journal_4", "df_journal", ["upper_title"])

df_final = Sqlite3Handler.read_df_from_sql(CONNECTION, """--sql
      SELECT DISTINCT
            bib.author
            ,bib.title
            ,bib.keywords
            ,bib.abstract
            ,bib.year
            ,bib.type_publication
            ,bib.doi
            ,bib.issn
            ,COALESCE(bib.journal, jor.upper_title) journal
            ,bib.source
            ,jor.scimago_value
            ,jor.jcs_value
      FROM 
            df_bibtex bib
            LEFT JOIN df_journal jor
                  ON (bib.issn = jor.issn_1
                  OR bib.issn = jor.issn_2
                  OR bib.issn = jor.issn_3
                  OR bib.upper_journal = jor.upper_title)
                  AND NOT (bib.issn is null and bib.upper_journal is null);
                  
""")

# Carga dos dados
OUTPUT_CONNECTION = Sqlite3Handler.get_connection(OUTPUT_DB_FILEPATH)

(df_final
      .pipe(sop.filter_column_by_regex, CONFIG)
      #.pipe(sop.export_dataframe, CONFIG, OUTPUT_FOLDER) // Anteriormente os dados precisavam ser salvos em arquivos, agora os dados serão carregados em um banco de dados SQLITE3
      .pipe(Sqlite3Handler.load_dataframe_to_db, 
            connection = OUTPUT_CONNECTION, 
            table_name = 't_bibtex_extraidos_manualmente', 
            if_exists = 'append')
)

# Transformação - API's IEEE e Science Direct

In [2]:

science_direct_responses = ScienceDirectApiHandler.get_data(CONFIG.get("science_direct_api_config", {}))
sd_api_df = ScienceDirectApiHandler.parse_response_to_dataframe(science_direct_responses)

ieee_responses = IeeeApiHandler.get_data(CONFIG.get("iee_api_config", {}))
ieee_api_df = IeeeApiHandler.parse_response_to_dataframe(ieee_responses).drop_duplicates()

OUTPUT_CONNECTION = Sqlite3Handler.get_connection(OUTPUT_DB_FILEPATH)

Sqlite3Handler.load_dataframe_to_db(sd_api_df, OUTPUT_CONNECTION, "t_api_science_direct", "append")
Sqlite3Handler.load_dataframe_to_db(ieee_api_df, OUTPUT_CONNECTION, "t_api_ieee", "append")


,authors,title,keywords,abstract,year,type_publication,doi,issn,journal,source
0,"Ikbal Taleb, Mohamed Adel Serhani, Rachida Dss...",Big Data Quality Assessment Model for Unstruct...,"Big Data, Data integrity, Data mining, Feature...",Big Data has gained an enormous momentum the p...,2018,Conferences,10.1109/INNOVATIONS.2018.8605945,2325-5498,IEEE,ieee API
1,"Sakda Loetpipatwanich, Preecha Vichitthamaros",Sakdas: A Python Package for Data Profiling an...,"Data integrity, Pipelines, Data visualization,...",Data Profiling and data quality management bec...,2020,Conferences,10.1109/IBDAP50342.2020.9245455,None,IEEE,ieee API
2,"Li Jin, Li Haosong, Xu Zhongping, Wang Ting, W...",Research on Wide-area Distributed Power Qualit...,"Power quality, Data integration, Distributed d...","With the advancement of the ""big operation"" sy...",2019,Conferences,10.1109/ICCCBDA.2019.8725668,None,IEEE,ieee API
3,"Xing Pan, Manli Zhang, Xi Chen",A Method of Quality Improvement Based on Big Q...,"Warranties, Data mining, Product design, Quali...",Quality warranty data includes big data of pro...,2018,Conferences,10.1109/QRS-C.2018.00115,None,IEEE,ieee API
4,"Andrew Burkhardt, Sheila Berryman, Ashley Brio...",Measuring Manufacturing Test Data Analysis Qua...,"Data integrity, Manufacturing, Measurement, De...",Manufacturing test data volumes are constantly...,2018,Conferences,10.1109/AUTEST.2018.8532518,1088-7725,IEEE,ieee API
...,...,...,...,...,...,...,...,...,...,...
395,"A.H. Moon, Ummer Iqbal Khan, Ashaq Hussain Dar...",Practical Implementation of WSN Based Data Acq...,"Wireless sensor networks, Data acquisition, Se...",WSN technology holds lot of promise in develop...,2013,Conferences,10.1109/ICMIRA.2013.21,None,IEEE,ieee API
396,"Chaang-Yung Kung, Pei-Yi Yang, Tzung-Ming Yan",Applying Grey Relational Method to Analyze the...,"Quality function deployment, Medical services,...",In the context of economic level sustaining it...,2006,Conferences,10.1109/ICSMC.2006.384482,1062-922X,IEEE,ieee API
397,"Tom´ Knap, Jan Michelfeit, Martin Necaský",Linked Open Data Aggregation: Conflict Resolut...,"Aggregates, Databases, Resource description fr...",The paradigm of publishing governmental data i...,2012,Conferences,10.1109/COMPSACW.2012.29,None,IEEE,ieee API
398,"Xuhui Chen, Jun Lu, Zhongyuan Liu",Assistance ontology of quality control for ent...,"Ontologies, Quality control, Data mining, Delt...",There are many quality domains in which ideas ...,2007,Conferences,10.1109/IEEM.2007.4419260,2157-362X,IEEE,ieee API
